# **Build App**  

### **Package Install**

- Using Steamlit 
- API docs : https://docs.streamlit.io/develop/api-reference

In [ ]:
# (1) streamlit 설치
!pip install streamlit -q

# (2) pyngrok 설치
!pip install pyngrok -q


print()
print(" ================== 👉👉👉👉👉  All Packages installed 👈👈👈👈👈 ===================\n")


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

file_path = "/content/gdrive/My Drive/Colab Notebooks/wtm/API/" #ngrok Token 

In [ ]:
from pyngrok import ngrok

file = open(file_path + "NGROK_Token.txt", "r")

ngrok_token = file.read()

file.close()

ngrok.set_auth_token(ngrok_token)


In [ ]:
!pip install konlpy -q

## **Web Source**

In [ ]:
%%writefile app.py

import streamlit as st


import numpy as np
from konlpy.tag import Okt
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


# ==============================================================================

# cf) reversed_word_index.txt


def sentiment_predict(new_sentence):
    okt = Okt()
    new_sentence = okt.nouns(new_sentence) # 토큰화

    stopwords = ['규정', '조항', '대한' ,'소원' ,'목적', '요지', '헌법소원심판', '필요', '사건', '위헌', '합헌', '법률', '제청', '청구인', '변호인', '제청', '재판', '피고인', '변호사', '심판', '청구' ,'재판관', '이유' ,'위헌확인', '피청구인', '수사' ,'이유', '원칙', '여부', '확인', '우리', '법률조항', '반대의견', '확인', '하나', '무엇', '제한', '인정', '대상', '입법', '헌법재판소', '재판소']
    new_sentence = [word for word in new_sentence if  (len(word) > 1) and (not word in stopwords)] # 불용어 제거

    tokenizer = Tokenizer()
    tokenizer.word_index = reversed_word_index
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩

    pad_new = pad_sequences(encoded, maxlen = 900) # 패딩

    loaded_model = load_model('/content/gdrive/My Drive/Colab Notebooks/wtm/finalproj/best_model_CNN.h5') # my best model

    # 예측
    predictions = loaded_model.predict(pad_new)
    predicted_class = np.argmax(predictions)  # 가장 높은 확률을 가진 클래스

    class_labels = ['언론 등 정신적 자유에 관한 결정', '가족 · 노동 등 사회관계에 관한 결정', '절차적 기본권 및 형사관계에 관한 결정', '정치 · 선거관계에 관한 결정', '경제 · 재산권 · 조세관계에 관한 결정']

    return f"\n◎ '위 판결문은 {np.max(predictions) * 100:.2f}% 확률로 <{class_labels[predicted_class]}> 입니다.\n"


#=======UI========================================================================

st.title("⚖ 판결문 카테고리 분류하기 ⚖")


with st.form(key="myform", clear_on_submit=True): #텍스트는 form으로 받아옴
  text = st.text_area("❤️ 카테고리를 분류하고자 하는 판결문을 입력해주세요\n\n ( [사건개요] [심판대상조항] [결정의 주요내용]이 담기면 좋습니다. )", height=100, value="")
  submit = st.form_submit_button("📑 분류")

if submit:
  #st.info(text)
  with st.spinner("in progress..."):
    predict = sentiment_predict(text)   # 위 함수를 호출
    st.success(predict)   # 내용 Display


if st.button('Reset'):
  st.empty()


Overwriting app.py


## **App 실행**

In [85]:
import urllib

print("◎ My IP is:   ",
      urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


◎ My IP is:    34.46.193.90


In [86]:
# nohup : no hang up의 약자로 프로그램을 백그라운드에서 지속적으로 실행시킴

!nohup streamlit run app.py &


nohup: appending output to 'nohup.out'


In [87]:
# ngrok tunnel 설정 및 실행

url = ngrok.connect(addr='8501')

url


<NgrokTunnel: "https://b47a-34-46-193-90.ngrok-free.app" -> "http://localhost:8501">

## **📌📌📌 Process 실행 중지 📌📌📌**

In [79]:
# ngrok tunnel 실행 중지

ngrok.kill()


In [93]:
# streamlit run ap  process id 확인

!ps -ef | grep streamlit


root       17511       1  5 01:04 ?        00:00:43 /usr/bin/python3 /usr/local/bin/streamlit run ap
root       20810     198  0 01:17 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       20812   20810  0 01:17 ?        00:00:00 grep streamlit


In [82]:
# streamlit run ap process 실행 중지

!kill -9  11901
